In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_path="/kaggle/input/rgb-light-field-dataset/drive-download-20220730T213349Z-001/"
print(os.listdir(data_path))

In [ ]:
import imageio
import torch

num_lightfields=2 #9 images per light field for the first drive folder
num_cameras_x=3
num_cameras_y=3

# Store lightfields as N(index of light field), C(RGB channels), H(height), W
#(width), U(vertical camera index), V(horizontal camera index)
lightfields = torch.zeros(num_lightfields, 3, 1080, 1920, num_cameras_y, num_cameras_x)
print(f'Lightfield shape {lightfields.shape}')
for i in range(num_lightfields):
    for x in range(num_cameras_x):
        for y in range(num_cameras_y):
            view = torch.Tensor(imageio.imread(os.path.join(data_path, f'image{i}_x{x}_y{y}.png'))).permute(2,0,1)
            lightfields[i,:,:,:,y,x] = view


# Visualization

In [ ]:
import matplotlib.pyplot as plt
import time
#showing 9 images of the first lightfield at index 0

for x in range(num_cameras_x):
    for y in range(num_cameras_y):
        plt.imshow((lightfields[0,:,:,:,y,x]).permute(1,2,0).numpy()/255.0)
        plt.show()
        time.sleep(0.5)


In [ ]:
class LightfieldDataset(torch.utils.data.Dataset):
    def __init__(self, data_path='', num_lightfields=2):
        self.num_lightfields=num_lightfields
        self.num_cameras_x=3
        self.num_cameras_y=3
        self.data_path=data_path
        
    def __len__(self):
        return self.num_lightfields

    def __getitem__(self, idx): #idx is index of lightfield (0,...,self.num_lightfields-1)
        lightfield = torch.zeros(3, 1080, 1920, self.num_cameras_y, self.num_cameras_x)
        for x in range(self.num_cameras_x):
            for y in range(self.num_cameras_y):
                view = torch.Tensor(imageio.imread(os.path.join(self.data_path, f'image{idx}_x{x}_y{y}.png'))).permute(2,0,1)
                lightfield[:,:,:,y,x] = view
        return lightfield
    
    
#https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [ ]:
train_data_path = "/kaggle/input/rgb-light-field-dataset/drive-download-20220730T213349Z-001/"
train_dataset = torch.utils.data.DataLoader(LightfieldDataset(data_path=train_data_path, num_lightfields=2), shuffle=True, batch_size=2)

In [ ]:
print(len(train_dataset))
for lf in train_dataset:
    print(lf.shape)

Currently, we have loaded in the train dataset. We plan to create 3 separate paths later on - train, test, and validation.